# Code Error Explainer

While an earlier project (03a) contained a bot explaining code snippets, in this project I created a bot explaining code errors.

So the intent is to have a teaching assistant explaining Python errors.

Scope:
- Explain what an error means
- Explain why it happens
- Suggest how to fix it conceptually

Explicitly out of scope:
- Generating runnable code
- Providing full solutions
- Rewriting code snippets

In [ ]:
# Imports

from google import genai
from dotenv import load_dotenv
import os


In [ ]:
# Load API key
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

if not API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment variables")

# Initialize client
client = genai.Client(api_key=API_KEY)


In [ ]:
# Audience mapping
audience_levels = {
    "1": "beginner",
    "2": "intermediate",
    "3": "developer"
}

audience_descriptions = {
    "1": "Explains in simple, plain language with analogies. Assumes no prior Python knowledge.",
    "2": "Uses some technical terms, assumes familiarity with Python basics.",
    "3": "Concise, technical, assumes programming experience and understanding of concepts."
}

print("Audience options:")
for num, desc in audience_descriptions.items():
    print(f"{num} = {audience_levels[num]}: {desc}")


This will be the prompt:

```
You are a Python teaching assistant.

A student ran the following Python code and received an error.

Your task:
- Explain what the error means
- Explain why it happened
- Explain how the student can fix it conceptually

Constraints (must follow):
- Do NOT provide any runnable code
- Do NOT rewrite the code
- Do NOT show full examples
- Do NOT include code blocks or syntax

Adapt your explanation to the student's level: {audience}

Code & Error:
{code_and_error}
```


In [ ]:
# Prompt
PROMPT_TEMPLATE = """
You are a Python teaching assistant.

A student ran the following Python code and received an error.

Your task:
- Explain what the error means
- Explain why it happened
- Explain how the student can fix it conceptually

Constraints (must follow):
- Do NOT provide any runnable code
- Do NOT rewrite the code
- Do NOT show full examples
- Do NOT include code blocks or syntax

Adapt your explanation to the student's level: {audience}

Code & Error:
{code_and_error}
"""


In [ ]:
# LLM call function
def explain_error(code_and_error: str, audience: str) -> str:
    prompt = PROMPT_TEMPLATE.format(
        code_and_error=code_and_error,
        audience=audience
    )

    # Model can be revised when required - had to do it mid exercise as the one indicated resource exhausted
    response = client.models.generate_content(
        model="models/gemini-2.5-flash-lite",
        # model="models/gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [ ]:
# Leak detection - look for code patterns or keywords
# to check that the output did not break the rules of providing full code iso only the concept to fix it
def contains_code_leak(text: str) -> bool:
    forbidden_patterns = [
        "```",
        "def ",
        "import ",
        "="
    ]
    return any(pat in text for pat in forbidden_patterns)


In [ ]:
# One-run interactive bot with retry on leak detection - max time of retries can be adapted here
def run_code_explainer_bot_once(max_retries: int = 5):
    print("\nWelcome to Python Error Explainer Bot!")
    print("Type 'quit' or 'exit' at any prompt to stop the bot.\n")
    
    # Step 1 — get combined input
    user_input = input(
        "Paste your code & error traceback here:\n"
    )
    if user_input.lower() in {"quit", "exit"}:
        print("Exiting bot... Goodbye!")
        return False

    # Step 2 — choose audience by number
    print("\nChoose audience level (type the number, or 'quit' to exit):")
    for num, desc in audience_descriptions.items():
        print(f"{num} = {audience_levels[num]}: {desc}")

    user_choice = ""
    while user_choice not in audience_levels:
        user_choice = input("Audience number: ").strip()
        if user_choice.lower() in {"quit", "exit"}:
            print("Exiting bot... Goodbye!")
            return False
        if user_choice not in audience_levels:
            print("Invalid choice. Please enter 1, 2, or 3.")

    user_audience = audience_levels[user_choice]

    # Step 3 — show input
    print("\n--- User Input ---")
    print(user_input)
    print("Audience level:", user_audience)

    # Step 4 — generate explanation with retry if code leak detected
    for attempt in range(1, max_retries + 1):
        explanation = explain_error(user_input, user_audience)
        if contains_code_leak(explanation):
            print(f"\n⚠️ Attempt {attempt}: Generated explanation contained code. Retrying...")
        else:
            print("\n--- Explanation ---")
            print(explanation)
            break
    else:
        print("\n⚠️ Could not generate explanation without code after several retries.")
        print("Please try again or simplify your input.")

    return True


In [ ]:
# Loop for optional repeated runs
while True:
    continue_bot = run_code_explainer_bot_once()
    if not continue_bot:
        break

    again = input("\nDo you want to run another explanation? (y/n): ").lower()
    if again not in {"y", "yes"}:
        print("Goodbye! Thanks for using the Python Error Explainer Bot.")
        break


## Reflection on this Code Explainer Bot Exercise

* **Combining inputs efficiently:** Using a single combined input for code and error/traceback simplified the prompt design and user workflow. Originally I had inputs for code and error separately which made it more complicated for the user

* **Prompt engineering matters:** Structuring prompts clearly with instructions on conceptual explanations and avoiding full code solutions improved response quality and reliability.

* **Dynamic parameters and user interaction:** Adding a dynamic choice for audience levels (with numeric codes) made the bot flexible while maintaining a simple interface. Building an optional loop for repeated runs reinforced control over the workflow.

* **Code safety and output handling:** Implementing **code leak detection** demonstrated how to safeguard against unintended code execution or exposure in outputs.

* **Iterative testing and refinement:** Running multiple test cases and observing responses highlighted the iterative nature of working with generative AI. It emphasized debugging prompts, retry strategies, and visual presentation in a notebook environment.

* **Model limitations and switching:** During testing, I noticed that the initially used model eventually failed to respond. Switching to another model, or using retry logic, ensured continuity and better-quality explanations. It reinforced the need to consider model choice, reliability, and fallback strategies when building interactive AI tools.

* **Further Improvements:** I was looking at wrapping of the input/output text, but would rather implement a VSCode extension than having additional code for such.

In [ ]:
# TEST SCENARIOS CELL (audience hardcoded per case)

test_cases = [
    {
        "name": "NameError",
        "audience": "beginner",
        "input": """---------------------------------------------------------------------------
NameError                               Traceback (most recent call last)
Cell [1], line 1
    print(x)
NameError: name 'x' is not defined"""
    },
    {
        "name": "TypeError",
        "audience": "beginner",
        "input": """---------------------------------------------------------------------------
TypeError                               Traceback (most recent call last)
Cell [2], line 3
    len(5)
TypeError: object of type 'int' has no len()"""
    },
    {
        "name": "IndexError",
        "audience": "intermediate",
        "input": """---------------------------------------------------------------------------
IndexError                               Traceback (most recent call last)
Cell [3], line 2
    lst = [1,2]
    print(lst[5])
IndexError: list index out of range"""
    },
    {
        "name": "ModuleNotFoundError",
        "audience": "intermediate",
        "input": """---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
Cell [4], line 1
    import some_nonexistent_module
ModuleNotFoundError: No module named 'some_nonexistent_module'"""
    },
    {
        "name": "ZeroDivisionError",
        "audience": "developer",
        "input": """---------------------------------------------------------------------------
ZeroDivisionError                         Traceback (most recent call last)
Cell [5], line 2
    x = 5/0
ZeroDivisionError: division by zero"""
    }
]

# Run test cases
for case in test_cases:
    print("\n" + "=" * 80)
    print(f"Test Case: {case['name']}")
    print(f"Audience: {case['audience']}")
    print("=" * 80)

    print("\n--- User Input ---")
    print(case["input"])

    explanation = explain_error(case["input"], case["audience"])

    print(f"\n--- Explanation ({case['audience']}) ---")
    print(explanation)
